# LSTM 예제

In [63]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

In [64]:
time_series = pd.DataFrame()
target_label = pd.DataFrame()

In [65]:
path = './../open/meta_data/train'
count = 0
row = pd.DataFrame()

for i in range(75):
    if i + 1 < 10:
        i_str = '0' + str(i +1)
    elif i + 1 > 10:
        i_str = str(i +1)
    elif i + 1 == 10:
        i_str = str(i + 1)

    file_list = os.listdir(path + '/CASE' + i_str)
    file_list_py = [file for file in file_list if file.endswith('.csv')]

    for l in file_list_py:
        data = pd.read_csv(path + '/CASE' + i_str + '/' + l)
        time_series = pd.concat([time_series, data])

In [66]:
path = './../open/meta_data/target'
count2 = 0

for j in range(75):
    if j + 1 < 10:
        j_str = '0' + str(j +1)
    elif j + 1 > 10:
        j_str = str(j +1)
    elif j + 1 == 10:
        j_str = str(j + 1)

    file_list = os.listdir(path + '/CASE' + j_str)
    file_list_py = [file for file in file_list if file.endswith('.csv')]

    for k in file_list_py:
        target = pd.read_csv(path + '/CASE' + j_str + '/' + k)
        target_label = pd.concat([target_label, target])

In [68]:
print(len(target_label))
print(len(time_series))

1292
1860480


In [101]:
target_dataset = target_label.to_numpy()
target_dataset = target_dataset[:, 1]

In [97]:
time_dataset = time_series.to_numpy()
time_dataset = time_dataset[:, 1:]

In [104]:
def sliding_windows(data, target):
    x = []
    y = []

    for i in range(len(target)):
        _x = data[(i * 1440): ((i + 1) * 1440)]
        _y = target[i]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

sc = MinMaxScaler()
training_data = sc.fit_transform(time_dataset)
target_data = sc.fit_transform(target_dataset.reshape(-1, 1))


x, y = sliding_windows(training_data, target_data)

train_size = int(len(y) * 0.7)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

In [109]:
testY.size()

torch.Size([388, 1])

In [ ]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()

        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)

        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))

        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))

        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))

        h_out = h_out.view(-1, self.hidden_size)

        out = self.fc(h_out)

        return out